# Power Flow Benchmark

This notebook compares the power flow results using a 10k bus system,
across different power flow solvers including: pandapower, PYPOWER, andes, and ams

In [1]:
import sys
import datetime
import importlib

import pandas as pd

import andes
import ams

import pandapower as pdp
import pypower.api as pyp
import pypsa as psa

In [2]:
print("Python version:", sys.version)
print("Last run time:", datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"))

for pkg in ['numba', 'pypower', 'pandapower', 'pypsa',
            'andes', 'ltbams']:
    print(f'{pkg}: {importlib.metadata.version(pkg)}')

Python version: 3.12.0 | packaged by conda-forge | (main, Oct  3 2023, 08:36:57) [Clang 15.0.7 ]
Last run time: 2024-12-06 11:00:42
numba: 0.60.0
pypower: 5.1.16
pandapower: 2.14.8
pypsa: 0.32.0
andes: 1.9.2
ltbams: 0.9.12


## ANDES

In [3]:
andes.config_logger(stream_level=30)

In [4]:
case_xlsx = './case_ACTIVSg10k.xlsx'
sa = andes.load(case_xlsx,
                setup=True, no_output=True, default_config=True,)

In [5]:
sa.PFlow.run(method='NR')

True

## AMS

***Note***: In AMS v0.9.13, the power flow is solved by ANDES.

In [6]:
ams.config_logger(stream_level=30)

In [7]:
sp = ams.load(case_xlsx,
              setup=True, no_output=True, default_config=True)

In [8]:
sp.PFlow.run()

Parsing OModel for <PFlow>
Building system matrices
Evaluating OModel for <PFlow>
Finalizing OModel for <PFlow>


True

## PYPOWER

In [9]:
ppc = andes.io.matpower.system2mpc(sa)

ppopt = pyp.ppoption(VERBOSE=0, OUT_ALL=0, PF_ALG=1)
ppc_sol, _ = pyp.runpf(ppc, ppopt)

ppc_bus = ams.shared.ppc2df(ppc_sol, model='bus')
ppc_bus.head()

,bus_i,type,pd,qd,gs,bs,area,vm,va,baseKV,zone,vmax,vmin
0,10001.0,1.0,9.64,2.54,0.0,0.0,0.0,1.009725,-41.506068,138.0,0.0,1.1,0.9
1,10002.0,1.0,15.98,6.04,0.0,0.0,0.0,1.015283,-38.702029,138.0,0.0,1.1,0.9
2,10003.0,1.0,14.39,1.09,0.0,0.0,0.0,1.030120,-35.694467,138.0,0.0,1.1,0.9
3,10004.0,1.0,0.00,0.00,0.0,0.0,0.0,1.031171,-35.581671,138.0,0.0,1.1,0.9
4,10005.0,1.0,10.45,2.75,0.0,0.0,0.0,1.031119,-35.587978,138.0,0.0,1.1,0.9


## pandapower

In [10]:
ppn = pdp.converter.from_ppc(ppc, f_hz=sp.config.freq)
pdp.runpp(ppn, algorithm='nr')
ppn.res_bus.head()

/Users/jinningwang/work/miniconda3/envs/moky/lib/python3.12/site-packages/pandapower/create.py:5049: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  net[table] = pd.concat([net[table], dd[dd.columns[~dd.isnull().all()]]], sort=False)
There are 4 branches which are considered as trafos - due to ratio unequal 0 or 1 - but connect same voltage levels.


,vm_pu,va_degree,p_mw,q_mvar
10001,1.024110,-43.243662,9.64,2.54
10002,1.014629,-40.388503,15.98,6.04
10003,1.019593,-37.319459,14.39,1.09
10004,1.020161,-37.207082,0.00,0.00
10005,1.020109,-37.213526,10.45,2.75


## PyPSA

PyPSA power flow failed for this case.

In [11]:
psn = psa.Network()
psn.import_from_pypower_ppc(ppc, overwrite_zero_s_nom=True)

psn_sol = psn.pf()
psn_sol['converged']

/Users/jinningwang/work/miniconda3/envs/moky/lib/python3.12/site-packages/pypsa/io.py:1471: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'True' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  pdf["branches"].loc[zero_s_nom, "s_nom"] = overwrite_zero_s_nom
/Users/jinningwang/work/miniconda3/envs/moky/lib/python3.12/site-packages/pypsa/io.py:1106: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.fillna(attrs["default"].to_dict())
INFO:pypsa.pf:Performing non-linear load-flow on AC sub-network <pypsa.components.SubNetwork object at 0x3607e22d0> for snapshots Index(['now'], dtype='object', name='snapshot')
/Users/jinningwang/work/miniconda3/envs

SubNetwork,0
snapshot,
now,False


## Comparasion

In [12]:
res = pd.DataFrame()

# voltage amplitude
bus_idx = sa.Bus.idx.v
res['bus'] = bus_idx

res['v_andes'] = sa.Bus.get(src='v', attr='v', idx=bus_idx)
res['v_ams'] = sp.Bus.get(src='v', attr='v', idx=bus_idx)
res['v_pyp'] = ppc_sol['bus'][:, 7]
res['v_pdp'] = ppn.res_bus['vm_pu'].values

# voltage angle
res['a_andes'] = sa.Bus.get(src='a', attr='v', idx=bus_idx) * andes.shared.rad2deg
res['a_ams'] = sp.Bus.get(src='a', attr='v', idx=bus_idx) * andes.shared.rad2deg
res['a_pyp'] = ppc_sol['bus'][:, 8]
res['a_pdp'] = ppn.res_bus['va_degree'].values

# calibrate using slack bus
slackbus_uid = sa.Bus.idx2uid(sa.Slack.bus.v)[0]
a_cols = ['a_andes', 'a_ams', 'a_pyp', 'a_pdp']
res[a_cols] = res[a_cols] - res[a_cols].iloc[slackbus_uid]

Here we randomly pick some rows.
The random seed is fixed to ensure the reproducibility.

In [13]:
res.sample(n=30, random_state=42)

,bus,v_andes,v_ams,v_pyp,v_pdp,a_andes,a_ams,a_pyp,a_pdp
6252,30323,1.013800,1.013800,1.013800,1.013800,-13.878608,-13.878608,-13.878170,-14.914052
4684,26079,0.993884,0.993884,0.993884,0.982512,-28.199874,-28.199874,-28.199357,-29.365947
1731,13649,1.020301,1.020301,1.020301,1.021014,19.317073,19.317073,19.317698,17.828975
4742,26137,1.024895,1.024895,1.024895,0.987563,-30.997540,-30.997540,-30.997018,-32.227182
4521,25916,1.040500,1.040500,1.040500,1.040500,-30.997406,-30.997406,-30.996878,-32.613015
6340,30411,1.029800,1.029800,1.029800,1.029800,-7.640517,-7.640517,-7.640090,-8.402686
576,10577,1.026131,1.026131,1.026131,1.023285,26.876740,26.876740,26.877344,25.386238
5202,28358,1.020587,1.020587,1.020587,0.983390,-30.029816,-30.029816,-30.029332,-31.301340
6363,30434,1.019200,1.019200,1.019200,1.028380,-13.411032,-13.411032,-13.410593,-14.506342
439,10440,1.020610,1.020610,1.020611,0.994534,12.210169,12.210169,12.210800,10.443540
